In [1]:
from sympy.physics.mechanics import (
    dynamicsymbols, ReferenceFrame, Point, RigidBody, Particle, System, Force
)
from sympy import zeros, symbols
import sympy
import numpy as np
import sympy as sp
import sympy.physics.mechanics as me
from IPython.display import display
from suspycious import Model
import suspycious.components as scmp
from pathway import LinearPathway
from utils import add_points, get_tension_dirs, give_deltas, give_tensions, get_linpaths, get_wire_dir, apply_force
from utils import (
    add_points, make_wire_bodies, get_points, get_deltas, get_linpaths,
    get_wires, get_forces, deltas_dict, linpaths_dict, add_wire_info,
    get_force_wire, apply_force_on_body, apply_gravity
)

In [2]:
d1, d2, n1, n2, w1, w2 = symbols('d1 d2 n1 n2 w1 w2', real=True, positive=True)
d3, n3, w3 = symbols('d3 n3 w3', real=True, positive=True)
d4, n4, w4 = symbols('d4 n4 w4', real=True, positive=True)
d5, n5, w5 = symbols('d5 n5 w5', real=True, positive=True)
d6, n6, w6 = symbols('d6 n6 w6', real=True, positive=True)
d7, n7, w7 = symbols('d7 n7 w7', real=True, positive=True)
d8, d9 = symbols('d8 d9', real=True, positive=True)

l1, l2, l3, l4, l2a = symbols('l1 l2 l3 l4 l_2a', positive=True)

In [3]:
def set_up_bodies():
    model2 = Model()
    S = model2.add(scmp.RigidBody("S"))
    B = model2.add(scmp.RigidBody("B"))
    C = model2.add(scmp.RigidBody("C"))
    D = model2.add(scmp.RigidBody("D"))
    F = model2.add(scmp.RigidBody("F"))
    
    S.set_global_position(0,0,0)
    B.set_global_position(0,0, -l1-d1)
    C.set_global_position(0,0, -l1 -d1-d2-l2-d3)
    D.set_global_position(0,0, -l1 -d1-d2-l2-d3-d4-l3-d5)
    F.set_global_position(0,0, -l1 -d1-d2-l2-d3-d4-l3-d5-d6-l4-d7)
    S.set_global_orientation(0,0,0)
    B.set_global_orientation(0,0,0)
    C.set_global_orientation(0,0,0)
    D.set_global_orientation(0,0,0)
    F.set_global_orientation(0,0,0)
    
    add_points(body=S, point='P', attachment_points=[w1, n1, 0])
    
    add_points(body=B, point='B', attachment_points=[w1, n1, d1])
    add_points(body=B, point='C', attachment_points=[w2, n2, -d2])
    
    add_points(body=C, point='D', attachment_points=[w2, n2, d3])
    add_points(body=C, point='E', attachment_points=[w3, n3, -d4])
    
    add_points(body=D, point='F', attachment_points=[w3, n3, d5])
    add_points(body=D, point='G', attachment_points=[w4, n4, -d6])
    
    add_points(body=F, point='H', attachment_points=[w4, n4, d7])
    add_points(body=F, point='J', attachment_points=[w5, n5, -d8])

    return S, B, C, D, F, model2

In [4]:
S, B, C, D, F, model2 = set_up_bodies()

/Users/siddharth/Desktop/suspycious/src/suspycious/components/body.py:81: SymPyDeprecationWarning: 

Support for the Body class has been removed, as its functionality is
fully captured by RigidBody and Particle.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-mechanics-body-class
for details.

This has been deprecated since SymPy version 1.13. It
will be removed in a future version of SymPy.

  self.__body = mech.Body(


In [5]:
sys = model2.system

In [6]:
#model2.add_bodies_system()

In [7]:
sys.bodies

()

In [8]:
n_violin_modes = 0
bodies_ = [[S, B], [B, C], [C, D], [D, F]]
attach_points_ = [[S.P1, B.B1], [B.C1, C.D1], [C.E1, D.F1], [D.G1, F.H1]]

In [9]:
points_bodies = get_points(
        n=n_violin_modes, num_blocks=4, model=model2,
        bodies=bodies_, attach_points=attach_points_, suspension_body=S)

model2.add_bodies_system()


In [10]:
sys.bodies

(Body('S_bdy', masscenter=S.com, frame=S, mass=S.M, inertia=Inertia(dyadic=S.I_{xx}*(S.x|S.x) + S.I_{xy}*(S.x|S.y) + S.I_{zx}*(S.x|S.z) + S.I_{xy}*(S.y|S.x) + S.I_{yy}*(S.y|S.y) + S.I_{yz}*(S.y|S.z) + S.I_{zx}*(S.z|S.x) + S.I_{yz}*(S.z|S.y) + S.I_{zz}*(S.z|S.z), point=S.com)),
 Body('B_bdy', masscenter=B.com, frame=B, mass=B.M, inertia=Inertia(dyadic=B.I_{xx}*(B.x|B.x) + B.I_{xy}*(B.x|B.y) + B.I_{zx}*(B.x|B.z) + B.I_{xy}*(B.y|B.x) + B.I_{yy}*(B.y|B.y) + B.I_{yz}*(B.y|B.z) + B.I_{zx}*(B.z|B.x) + B.I_{yz}*(B.z|B.y) + B.I_{zz}*(B.z|B.z), point=B.com)),
 Body('C_bdy', masscenter=C.com, frame=C, mass=C.M, inertia=Inertia(dyadic=C.I_{xx}*(C.x|C.x) + C.I_{xy}*(C.x|C.y) + C.I_{zx}*(C.x|C.z) + C.I_{xy}*(C.y|C.x) + C.I_{yy}*(C.y|C.y) + C.I_{yz}*(C.y|C.z) + C.I_{zx}*(C.z|C.x) + C.I_{yz}*(C.z|C.y) + C.I_{zz}*(C.z|C.z), point=C.com)),
 Body('D_bdy', masscenter=D.com, frame=D, mass=D.M, inertia=Inertia(dyadic=D.I_{xx}*(D.x|D.x) + D.I_{xy}*(D.x|D.y) + D.I_{zx}*(D.x|D.z) + D.I_{xy}*(D.y|D.x) + D.I_{yy

In [11]:
# def get_deltas(points_body, model, suspension_body):
#     n = len(points_body['points']) // 4 - 1
#     deltas_ = []
#     op_point = model.op_point
#     sb = suspension_body
#     deltas_a = {}
#     top_body_name = points_body['bodies'][0].name
#     bottom_body_name = points_body['bodies'][-1].name
#     delta_body_str = [
#         f'del_{top_body_name}_{bottom_body_name}_{i+1}' 
#         for i in range(len(points_body['bodies']) - 4)
#     ]

#     for j in range(4):  # 4 because of 4 wires
#         for i in range(n):  # n is wire parts per wire (Depends on violin modes)
#             m = 4 * i + j
#             attach_points = (points_body['points'][m], points_body['points'][m+4])
#             print(attach_points)
#             delta = [
#                 attach_points[0].pos_from(attach_points[1])
#                 .express(sb.global_frame)
#                 .magnitude() -
#                 attach_points[0].pos_from(attach_points[1])
#                 .express(sb.global_frame)
#                 .subs(op_point).magnitude()
#             ]
#             deltas_.append(delta)

#     deltas_unwrapped = [item for items in deltas_ for item in items]
#     dict_del_body = dict(zip(delta_body_str, deltas_unwrapped))

#     return dict_del_body


# def deltas_dict(points_body, num_blocks, model, suspension_body):
#     delta_ = {}

#     for i in range(num_blocks):
#         top_body_name = points_body[f'body{i+1}']['bodies'][0].name
#         bottom_body_name = points_body[f'body{i+1}']['bodies'][-1].name
#         print(f"Calculating deltas for points between bodies {top_body_name} and {bottom_body_name}")
#         delta_[f'body{i+1}'] = get_deltas(
#             points_body=points_body[f'body{i+1}'], 
#             model=model, 
#             suspension_body=suspension_body
#         )

#     return delta_

In [12]:
points_bodies

{'body1': {'points': [S.P1, S.P2, S.P3, S.P4, B.B1, B.B2, B.B3, B.B4],
  'bodies': [<suspycious.components.body.RigidBody at 0x10f09efd0>,
   <suspycious.components.body.RigidBody at 0x1450e7dc0>]},
 'body2': {'points': [B.C1, B.C2, B.C3, B.C4, C.D1, C.D2, C.D3, C.D4],
  'bodies': [<suspycious.components.body.RigidBody at 0x1450e7dc0>,
   <suspycious.components.body.RigidBody at 0x10f09e130>]},
 'body3': {'points': [C.E1, C.E2, C.E3, C.E4, D.F1, D.F2, D.F3, D.F4],
  'bodies': [<suspycious.components.body.RigidBody at 0x10f09e130>,
   <suspycious.components.body.RigidBody at 0x145199340>]},
 'body4': {'points': [D.G1, D.G2, D.G3, D.G4, F.H1, F.H2, F.H3, F.H4],
  'bodies': [<suspycious.components.body.RigidBody at 0x145199340>,
   <suspycious.components.body.RigidBody at 0x1451e1940>]}}

In [13]:
deltas = deltas_dict(
    points_body=points_bodies, num_blocks=4, model=model2,
    suspension_body=S
)

linpaths = linpaths_dict(
    points_body=points_bodies, num_blocks=4, model=model2
)

num_blocks = 4
all_masses = [
    i.body.mass for j in range(num_blocks)
    for i in points_bodies[f'body{j + 1}']['bodies']
]

all_masses_ = list(dict.fromkeys(all_masses))
wires = get_wires(
    num_blocks=4, point_body=points_bodies,
    deltas=deltas, linpaths=linpaths
)

ks = symbols('k1:5')

Calculating deltas for points between bodies S and B
(S.P1, B.B1)
(S.P2, B.B2)
(S.P3, B.B3)
(S.P4, B.B4)
Calculating deltas for points between bodies B and C
(B.C1, C.D1)
(B.C2, C.D2)
(B.C3, C.D3)
(B.C4, C.D4)
Calculating deltas for points between bodies C and D
(C.E1, D.F1)
(C.E2, D.F2)
(C.E3, D.F3)
(C.E4, D.F4)
Calculating deltas for points between bodies D and F
(D.G1, F.H1)
(D.G2, F.H2)
(D.G3, F.H3)
(D.G4, F.H4)
Finding linpath between S.P1 and B.B1
Finding linpath between S.P2 and B.B2
Finding linpath between S.P3 and B.B3
Finding linpath between S.P4 and B.B4
Finding linpath between B.C1 and C.D1
Finding linpath between B.C2 and C.D2
Finding linpath between B.C3 and C.D3
Finding linpath between B.C4 and C.D4
Finding linpath between C.E1 and D.F1
Finding linpath between C.E2 and D.F2
Finding linpath between C.E3 and D.F3
Finding linpath between C.E4 and D.F4
Finding linpath between D.G1 and F.H1
Finding linpath between D.G2 and F.H2
Finding linpath between D.G3 and F.H3
Finding li

In [14]:
deltas.keys()

dict_keys(['body1', 'body2', 'body3', 'body4'])

In [15]:
all_masses_

[S.M, B.M, C.M, D.M, F.M]

In [16]:
n_wire_per_wire = n_violin_modes + 1
body1_force = get_forces(
        n_body=1, n_wire_body=n_wire_per_wire,
        wire_dict=wires, spring_constants=ks, masses=all_masses_
    )
body2_force = get_forces(
    n_body=2, n_wire_body=n_wire_per_wire,
    wire_dict=wires, spring_constants=ks, masses=all_masses_
)
body3_force = get_forces(
    n_body=3, n_wire_body=n_wire_per_wire,
    wire_dict=wires, spring_constants=ks, masses=all_masses_
)
body4_force = get_forces(
    n_body=4, n_wire_body=n_wire_per_wire,
    wire_dict=wires, spring_constants=ks, masses=all_masses_
)

In [17]:
body1_force.keys()

dict_keys(['wire11', 'wire12', 'wire13', 'wire14'])

In [18]:
#body1_force['wire11']['force']

In [18]:
#body1_force['wire12']['force']

In [51]:
# F = symbols('F')
# body1_force['wire21']['path'].to_loads(F)[1]

In [19]:
# for i in body1_force.keys():
#     force = body1_force[i]['force']
#     path = body1_force[i]['path']
#     points = body1_force[i]['points']
#     bodies = body1_force[i]['bodies']

#     sys.add_loads(path.to_loads(-force, frame=S.global_frame)[0])
#     sys.add_loads(path.to_loads(-force, frame=S.global_frame)[1])

In [18]:
for j in [body1_force, body2_force, body3_force, body4_force]:
    for i in j.keys():
        force = j[i]['force']
        path = j[i]['path']
        points = j[i]['points']
        bodies = j[i]['bodies']
    
        sys.add_loads(path.to_loads(-force, frame=S.global_frame)[0])
        sys.add_loads(path.to_loads(-force, frame=S.global_frame)[1])

In [19]:
len(sys.loads)

32

In [20]:
def apply_gravity(body):
    g = symbols('g')
    body.Fz = -g*body.body.mass
         
    return

In [21]:
type(model2.components['S'])

suspycious.components.body.RigidBody

In [22]:
sys.bodies[0]

Body('S_bdy', masscenter=S.com, frame=S, mass=S.M, inertia=Inertia(dyadic=S.I_{xx}*(S.x|S.x) + S.I_{xy}*(S.x|S.y) + S.I_{zx}*(S.x|S.z) + S.I_{xy}*(S.y|S.x) + S.I_{yy}*(S.y|S.y) + S.I_{yz}*(S.y|S.z) + S.I_{zx}*(S.z|S.x) + S.I_{yz}*(S.z|S.y) + S.I_{zz}*(S.z|S.z), point=S.com))

In [23]:
apply_gravity(model2.components['S'])
apply_gravity(model2.components['B'])
apply_gravity(model2.components['C'])
apply_gravity(model2.components['D'])
apply_gravity(model2.components['F'])

In [24]:
#sys.loads[1]#.force.dot(model2.frame.z).subs(model2.op_point)

In [25]:
#kanel = model2.extract_statespacesystem()
kane = model2.extract_statespacesystem()

In [26]:
Amat, Bmat = kane.A, kane.B

In [27]:
#kane.outputs

In [28]:
E = kane.E

In [27]:
Amat[40,10]

-4*d1*(-d1*g*(B.M/4 + C.M/4 + D.M/4 + F.M/4)/l1 - g*(B.M/4 + C.M/4 + D.M/4 + F.M/4)) + 4*d2*(d2*g*(C.M/4 + D.M/4 + F.M/4)/l2 + g*(C.M/4 + D.M/4 + F.M/4)) + 4*k1*w1**2 + 4*k2*w2**2

In [28]:
kane.outputs

{S.x(t): 0,
 S.y(t): 1,
 S.z(t): 2,
 S.theta_x(t): 3,
 S.theta_y(t): 4,
 S.theta_z(t): 5,
 B.x(t): 6,
 B.y(t): 7,
 B.z(t): 8,
 B.theta_x(t): 9,
 B.theta_y(t): 10,
 B.theta_z(t): 11,
 C.x(t): 12,
 C.y(t): 13,
 C.z(t): 14,
 C.theta_x(t): 15,
 C.theta_y(t): 16,
 C.theta_z(t): 17,
 D.x(t): 18,
 D.y(t): 19,
 D.z(t): 20,
 D.theta_x(t): 21,
 D.theta_y(t): 22,
 D.theta_z(t): 23,
 F.x(t): 24,
 F.y(t): 25,
 F.z(t): 26,
 F.theta_x(t): 27,
 F.theta_y(t): 28,
 F.theta_z(t): 29}

In [38]:
#kane.inputs

In [29]:
Bmat[40,4]

-4*d1*(-d1*g*(B.M/4 + C.M/4 + D.M/4 + F.M/4)/l1 - g*(B.M/4 + C.M/4 + D.M/4 + F.M/4)) + 4*d2*(d2*g*(C.M/4 + D.M/4 + F.M/4)/l2 + g*(C.M/4 + D.M/4 + F.M/4)) + 4*k1*w1**2 + 4*k2*w2**2

In [30]:
kane.kanel.r

Matrix([
[  B.delta_x(t)],
[  B.delta_y(t)],
[  B.delta_z(t)],
[B.epsilon_x(t)],
[B.epsilon_y(t)],
[B.epsilon_z(t)],
[  C.delta_x(t)],
[  C.delta_y(t)],
[  C.delta_z(t)],
[C.epsilon_x(t)],
[C.epsilon_y(t)],
[C.epsilon_z(t)],
[  D.delta_x(t)],
[  D.delta_y(t)],
[  D.delta_z(t)],
[D.epsilon_x(t)],
[D.epsilon_y(t)],
[D.epsilon_z(t)],
[  F.delta_x(t)],
[  F.delta_y(t)],
[  F.delta_z(t)],
[F.epsilon_x(t)],
[F.epsilon_y(t)],
[F.epsilon_z(t)],
[  S.delta_x(t)],
[  S.delta_y(t)],
[  S.delta_z(t)],
[S.epsilon_x(t)],
[S.epsilon_y(t)],
[S.epsilon_z(t)]])

In [36]:
#kanel.f_3.shape

In [37]:
#kanel.q

In [31]:
#kanel.f_3[10].diff(kanel.q[10]).subs(model2.op_point)

In [32]:
#kanel.f_3[10].diff(kanel.q[0]).subs(model2.op_point)

In [33]:
#Amat, Bmat = kanel.linearize(A_and_B=True, op_point=model2.op_point)

In [34]:
#sols = kanel.linearize(op_point=model2.op_point)

In [35]:
#sols[1][40,10]

In [36]:
#Amat.shape

In [37]:
#Amat[40,10]

In [37]:
#sys.mass_matrix

In [38]:
#sys.mass_matrix_full

In [39]:
#kanel.r

In [40]:
#kanel.u